In [8]:
import pandas as pd
import numpy as np
import os
import re
import json as js
from pathlib import Path
from tqdm import tqdm
from ast import literal_eval

In [1]:
POS = ["v","a","n","pron","adv","num","p"]

def is_contain_bold_and_italic(font):
    contains_bold = False; contains_italic = False
    for i in font:
        if "bold" in i.lower(): contains_bold = True
        if "italic" in i.lower(): contains_italic = True
        if contains_bold == True and contains_italic == True: return True
    return False

def is_contain_only_whitespaces(s):
    if re.match(r'^\s*$', str(s)): return True
    return False

def is_prakategorial(s):
    kata = s.strip()
    if len(kata) > 1:
        if is_contain_only_whitespaces(kata[-2]) and (kata[-1] in POS): return True
    else:
        if (kata[-1] in POS): return True
        
    if re.match(r'.*\,$',str(s)): return True
    return False

In [2]:
# kategorisasi entry, untuk memisahkan mana yang main entry dan sub entry
def categorize_main_entry(entries, posisi, pages):
    output = []
    
    i = 0; j = 0
    while i < len(pages):
        if isinstance(pages[i], list): # kasus entry cross page
            prev_posisi_x0 = posisi[i-1][0]
            
            if abs(posisi[i][0] - prev_posisi_x0) <= 3:
                output.append(output[i-1])
                
            else:
                batas_atas = round(prev_posisi_x0 + (prev_posisi_x0 * (2/100)),2) # error 2%
                batas_kolom = 2*batas_atas
                
                if posisi[i][0] > batas_atas and posisi[i][0] < batas_kolom:
                    output.append(0)
                    
                else:
                    output.append(1)
                    
            i += 1; j += 1
        
        else:   
            entries_by_page = []; posisi_by_page = []; curr_page = pages[j] 
        
            while curr_page == pages[i]: # kelompokkan entri berdasarkan halaman
                entries_by_page.append(entries[j])
                posisi_by_page.append(posisi[j][0])
                j += 1
                
                if j > len(pages) - 1: 
                    break
                    
                curr_page = pages[j]
                
            sorted_posisi = sorted(posisi_by_page) # urutkan                
            i = j; k = 0; l = 0 # update nilai i
            
            while k < len(posisi_by_page):
                if is_prakategorial(entries_by_page[k]):
                    if k == len(posisi_by_page)-1:
                        output.append(1); break
                    else:
                        output.append(1)
                        output.append(0)
                        k += 2
                else:
                    if abs(posisi_by_page[k] - sorted_posisi[l]) > 3:
                        output.append(0); k += 1 # handle header atau nomor halaman

                    else:
                        output.append(1) # index pertama setelah header atau nomor halaman
                        batas_atas = round(posisi_by_page[k] + (posisi_by_page[k] * (2/100)),2) # error 2%
                        batas_kolom = 2*batas_atas

                        m = k + 1
                        if m > len(posisi_by_page) - 1: break

                        nxt_posisi = posisi_by_page[m]
                        while nxt_posisi > batas_atas and nxt_posisi < batas_kolom:
                            output.append(0); m += 1

                            if m > len(posisi_by_page) - 1: 
                                break 

                            nxt_posisi = posisi_by_page[m]

                        k = m
                        if nxt_posisi < batas_kolom:
                            l += 1
                        else:
                            l = m
                
                
    return output 

In [11]:
directory_CSV = "4. Remove Anomali"
directory_hasil = "5. Kategorisasi Main Entry"

for filename in tqdm(os.listdir(directory_CSV)):
    print("====" + filename + "====")
    kamus = pd.read_csv(directory_CSV + "/" + filename)
    
    posisi_entry = []
    for i in kamus["posisi_entry"].values.tolist():
        posisi_entry.append(literal_eval(i))
        
    page = []
    for i in kamus["page"].values.tolist():
        if not isinstance(i,int):
            page.append(literal_eval(i))
        else:
            page.append(int(i))
            
    entries = kamus["Entri"].values.tolist()
            
    kategori = categorize_main_entry(entries, posisi_entry, page)
    kamus["main_entry"] = kategori
    
    new_filename = os.path.splitext(filename)[0]
    kamus.to_csv(directory_hasil + "/" + new_filename + "-WithMainEntry.csv",index=False)
    
#     input_fonts = data["font"].values.tolist()
    
#     if is_contain_bold_and_italic(input_fonts):
#         print("====" + new_filename + "====")
#         CSV_res = build_corpus_one_entry_by_font_pos(data)

#         result_csv = pd.DataFrame.from_dict(CSV_res)
#         result_csv = result_csv[result_csv["Entri"] != ""]
#         result_csv = result_csv.reset_index(drop=True)
#         result_csv.to_csv(directory_hasil + "/" + new_filename + "-one_entry_from_JSON_font_posisi.csv",index=False)
#         try:
#             CSV_res = build_corpus_one_entry_by_font_pos(data)

#             result_csv = pd.DataFrame.from_dict(CSV_res)
#             result_csv.to_csv(directory_hasil + "/" + new_filename + "-one_entry_from_JSON_font_posisi.csv",index=False)
#         except:
#             print("==== Kamus Gagal ====")
#             print(new_filename)

  0%|                                                                                           | 0/43 [00:00<?, ?it/s]

====10_Replace-categorizeAnomali.csv====


  2%|█▉                                                                                 | 1/43 [00:00<00:13,  3.06it/s]

====12_Replace-categorizeAnomali.csv====


  7%|█████▊                                                                             | 3/43 [00:01<00:13,  2.96it/s]

...

 93%|████████████████████████████████████████████████████████████████████████████▎     | 40/43 [00:13<00:00,  5.35it/s]

====87_Replace-categorizeAnomali.csv====


 98%|████████████████████████████████████████████████████████████████████████████████  | 42/43 [00:14<00:00,  4.44it/s]

====89_Replace-categorizeAnomali.csv====
====91_Replace-categorizeAnomali.csv====


100%|██████████████████████████████████████████████████████████████████████████████████| 43/43 [00:14<00:00,  2.93it/s]
